In [2]:
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install plotly
!pip install llamabot
import llamabot
!pip install tqdm
from tqdm import tqdm
from tqdm import tqdm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


...

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\michi\AppData\Roaming\Python\Python310\site-packages\llamabot\bot\__init__.py:24: RuntimeWarning: No OpenAI API key found. Please set OPENAI_API_KEY in your environment.
  warnings.warn(


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
df = pd.read_pickle('processed_data.pkl')
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [4]:
from typing import TypedDict
class Result(TypedDict):
    model:str
    time:datetime.datetime
    promt_hash: int
    hate_speach_probability:float
results:list[Result]=[]

In [22]:
prompt = """
You are Sally and your job is to be an AI Classifier for Hatespeach Detection.
You are known for your rigourous and unpartial judgement and have vast experience in the field of hate speach. 

You will be given a sentence from a forum and have to answer the question if this is hate speach.

Please exclusively respond in the following form to the question:
```json
{ 
  "relevant_facts": [
    "fact_1",
    "fact_2",
    "fact_3",
  ],
  "contained_explicit_hate_speech_vocabular": ["word1", "word2", "word3", "word4"],
  "argument_for_hate_speech": "short argument why this is hate speach",
  "argument_against_hate_speech": "short argument why this is not hate speach",
  "likelihood_of_presence_of_implicit_hate_speech": 0.17,
  "likelihood_of_presence_of_explicit_hate_speech": 0.79,
  "likelihood_of_ad_hominem_attack"; 013,
  "likelihood_of_minority_attack": 0.24,
  "likelyhood_of_takedown_on_social_media": 0.42,
  "hate_speech_probability": 0.42,
}
```

Please be carefull in your responses, as otherwise a citten will be killed and I will use my job. For good results, I will tip 1000€.
"""

In [32]:
df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
0,12834217_1,572066,1346,0,False,"As of March 13th , 2014 , the booklet had been...",False,False
1,12834217_2,572066,1346,0,False,In order to help increase the booklets downloa...,True,False
2,12834217_3,572066,1346,0,False,( Simply copy and paste the following text int...,False,False
3,12834217_4,572066,1346,0,True,Click below for a FREE download of a colorfull...,True,False
4,12834217_5,572066,1346,0,False,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,False,False
...,...,...,...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False,"Billy - `` That guy would n't leave me alone ,...",False,False
10940,33677019_1,735154,1388,0,False,Wish we at least had a Marine Le Pen to vote f...,False,False
10941,33677019_2,735154,1388,0,False,Its like the choices are white genocide candid...,False,False
10942,33677053_1,572266,1388,0,True,Why White people used to say that sex was a si...,True,False


In [36]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("PPLX_KEY")

url = "https://api.perplexity.ai/chat/completions"

results = []
for index, row in tqdm(testing_df.iterrows(), total=testing_df["text"].count()):
    payload = {
        "model": "mistral-7b-instruct",
        "messages": [
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": row["text"]
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    response = requests.post(url, json=payload, headers=headers)
    data = json.loads(response.text)
    data = data['choices'][0]['message']['content']
    results.append({
        "model": "mistral",
        "time": datetime.datetime.now(),
        "promt_hash": hash(prompt),
        "prompt": row["text"],
        "answer": data,
        "truth": row["hate"]
    })
    print(data)

dfres = pd.DataFrame(results)
dfres.to_pickle("mistral_results.pkl")

  0%|          | 1/478 [00:03<25:15,  3.18s/it]

{ 
  "relevant_facts": [
    "The speaker expresses a dislike for a specific neighborhood and its children.",
  ],
  "contained_explicit_hate_speech_vocabular": [
    "kids like that",
  ],
  "argument_for_hate_speech": "The speaker uses derogatory language to describe children, implying that they are disgusting.",
  "argument_against_hate_speech": "The speaker is merely expressing a personal preference and not directly targeting or threatening harm towards individuals or groups based on protected characteristics.",
  "likelihood_of_presence_of_implicit_hate_speech": 0.32,
  "likelihood_of_presence_of_explicit_hate_speech": 0.63,
  "likelihood_of_ad_hominem_attack": 0.11,
  "likelihood_of_minority_attack": 0,
  "likelyhood_of_takedown_on_social_media": 0.54,
  "hate_speech_probability": 0.63,



  0%|          | 2/478 [00:06<25:26,  3.21s/it]

{ 
  "relevant_facts": [
    "The sentence refers to white students at schools behaving in a certain way.",
  ],
  "contained_explicit_hate_speech_vocabular": [],
  "argument_for_hate_speech": "The sentence implies a negative generalization towards a specific racial group, which could be considered hate speech.",
  "argument_against_hate_speech": "The sentence does not include any explicitly hateful or violent language, and there is no clear intent to incite harm or violence.",
  "likelihood_of_presence_of_implicit_hate_speech": 0.36,
  "likelihood_of_presence_of_explicit_hate_speech": 0.00,
  "likelihood_of_ad_hominem_attack": 0.12,
  "likelihood_of_minority_attack": 0.63,
  "likelyhood_of_takedown_on_social_media": 0.51,
  "hate_speech_probability": 0.36,
}

Please note that this judgment is based on the given sentence alone, and context and additional information could change the classification


  1%|          | 3/478 [00:08<22:52,  2.89s/it]

...

```json
{ 
  "relevant_facts": ["The sentence contains stereotypes about Black people,"],
  "contained_explicit_hate_speech_vocabular": ["Blacks", "hate", "cold", "athletes", "running", "lions", "tigers"],
  "argument_for_hate_speech": "The sentence makes a derogatory statement about Black people as a whole, suggesting that they are inferior and only good at athletics due to a perceived need to escape danger from predators.",
  "argument_against_hate_speech": "There is no argument against hate speech in this sentence. It contains explicit hate speech towards Black people.",
  "likelihood_of_presence_of_implicit_hate_speech": 0,
  "likelihood_of_presence_of_explicit_hate_speech": 1,
  "likelihood_of_ad_hominem_attack": 0,
  "likelihood_of_minority_attack": 1,
  "likelyhood_of_takedown_on_social_media": 1,
  "hate_speech_probability": 1,
}
``


100%|█████████▉| 477/478 [25:16<00:02,  2.77s/it]

{ 
  "relevant_facts": [
    "The sentence discusses a belief about the filling of holes below 40,000 feet.",
  ],
  "contained_explicit_hate_speech_vocabular": [],
  "argument_for_hate_speech": "There is no apparent hate speech in this sentence.",
  "argument_against_hate_speech": "None directly related to this sentence.",
  "likelihood_of_presence_of_implicit_hate_speech": 0.01,
  "likelihood_of_presence_of_explicit_hate_speech": 0.0,
  "likelihood_of_ad_hominem_attack": 0.0,
  "likelihood_of_minority_attack": 0.0,
  "likelyhood_of_takedown_on_social_media": 0.01,
  "hate_speech_probability": 0.0



100%|██████████| 478/478 [25:18<00:00,  3.18s/it]

{ 
  "relevant_facts": [],
  "contained_explicit_hate_speech_vocabular": [],
  "argument_for_hate_speech": "There is no hate speech in the given sentence.",
  "argument_against_hate_speech": "The sentence expresses agreement and understanding.",
  "likelihood_of_presence_of_implicit_hate_speech": 0,
  "likelihood_of_presence_of_explicit_hate_speech": 0,
  "likelihood_of_ad_hominem_attack": 0,
  "likelihood_of_minority_attack": 0,
  "likelyhood_of_takedown_on_social_media": 0,
  "hate_speech_probability": 0,



In [37]:
testing_df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
21,12849464_3,575975,1346,1,True,I 've always considered teaching as one of the...,False,True
22,12849464_4,575975,1346,1,True,And the sad thing is the white students at tho...,False,True
34,12855497_1,575939,1346,0,False,This whole situation starts to become really b...,False,True
75,13063290_1,578990,1345,0,False,That's all I needed to hear .,False,True
92,13288494_1,572180,1345,6,False,Excellent Article !! * * * * * Why Were n't Th...,False,True
...,...,...,...,...,...,...,...,...
10708,30594322_1,577718,1362,0,False,I have one word for you ....... MOVE . We chos...,False,True
10777,30663322_3,578085,1348,0,False,There are only eight people living within a mi...,False,True
10816,30754294_1,575879,1354,0,True,you just say blacks hate the cold. the only re...,False,True
10856,31442389_3,572414,1346,0,False,"And all those holes below 40,000 feet are fill...",False,True


In [38]:
results

[{'model': 'mistral',
  'time': datetime.datetime(2024, 1, 8, 15, 30, 47, 202459),
  'promt_hash': -3450054670443982230,
  'prompt': "I 've always considered teaching as one of the professions I would like to get into , but not in a neighbourhood like that ... never. kids like that disgust me .",
  'answer': '{ \n  "relevant_facts": [\n    "The speaker expresses a dislike for a specific neighborhood and its children.",\n  ],\n  "contained_explicit_hate_speech_vocabular": [\n    "kids like that",\n  ],\n  "argument_for_hate_speech": "The speaker uses derogatory language to describe children, implying that they are disgusting.",\n  "argument_against_hate_speech": "The speaker is merely expressing a personal preference and not directly targeting or threatening harm towards individuals or groups based on protected characteristics.",\n  "likelihood_of_presence_of_implicit_hate_speech": 0.32,\n  "likelihood_of_presence_of_explicit_hate_speech": 0.63,\n  "likelihood_of_ad_hominem_attack": 0.1

In [39]:
testing_df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
21,12849464_3,575975,1346,1,True,I 've always considered teaching as one of the...,False,True
22,12849464_4,575975,1346,1,True,And the sad thing is the white students at tho...,False,True
34,12855497_1,575939,1346,0,False,This whole situation starts to become really b...,False,True
75,13063290_1,578990,1345,0,False,That's all I needed to hear .,False,True
92,13288494_1,572180,1345,6,False,Excellent Article !! * * * * * Why Were n't Th...,False,True
...,...,...,...,...,...,...,...,...
10708,30594322_1,577718,1362,0,False,I have one word for you ....... MOVE . We chos...,False,True
10777,30663322_3,578085,1348,0,False,There are only eight people living within a mi...,False,True
10816,30754294_1,575879,1354,0,True,you just say blacks hate the cold. the only re...,False,True
10856,31442389_3,572414,1346,0,False,"And all those holes below 40,000 feet are fill...",False,True
